# CSC 620 HA #11

By: Mark Kim

Adapted from: [Utham Bathoju](https://www.kaggle.com/code/uthamkanth/beginner-tf-idf-and-cosine-similarity-from-scratch/notebook)

This notebook is broken up into three sections as follows:

1. [Program_0](#program_0): Create a copy of the below notebook (or export it as
   python program), and add detailed description for each code block, in your
   own words.
2. [Program_1](#program_1): Revise the above program to replace the toy dataset
   with with a larger text dataset of your choice.
3. [Program_2](#program_2): Create a variation of Program_1 that uses only TF
   representation to compute the similarity between a query and document.
4. [Writeup](#Writeup): Submit a short write-up (1 to 2 paragraphs) that compares and contrasts the document rankings provided by Program_1 and Program_2 for the same 3 queries of your choice.  Reflect on why is the ranking different, which representation provides better ranking, etc.

## Program_0

Create a copy of the notebook and add detailed descriptions.

In [2]:
import math
import pandas as pd
import numpy as np

Define query and toy documents

In [3]:
#documents
doc1 = "I want to start learning to charge something in life"
doc2 = "reading something about life no one else knows"
doc3 = "Never stop learning"
#query string
query = "life learning"

## Raw term frequency counts

The following function was created simply to illustrate that term frequency is
not a good measure to base a search query from.  The terms have not been
normalized (e.g. capitalization removed, stopwords removed, etc.).  Furthermore,
we do not normalize to large term counts.  As we learned in lecture, a word that
appears 100 times should not weigh 100 times heavier in determining
"importance".  Lastly, for the data to be useful, we want to create
probabilities that terms appear, hence, raw counts will not be useful for us.

In [4]:
def compute_tf(docs_list):
    for doc in docs_list:
        doc1_lst = doc.split(" ")
        wordDict_1= dict.fromkeys(set(doc1_lst), 0)

        for token in doc1_lst:
            wordDict_1[token] +=  1
        df = pd.DataFrame([wordDict_1])
        idx = 0
        new_col = ["Term Frequency"]    
        df.insert(loc=idx, column='Document', value=new_col)
        print(df)
        
compute_tf([doc1, doc2, doc3])

         Document  want  something  learning  life  to  charge  in  start  I
0  Term Frequency     1          1         1     1   2       1   1      1  1
         Document  about  something  no  reading  else  life  one  knows
0  Term Frequency      1          1   1        1     1     1    1      1
         Document  stop  learning  Never
0  Term Frequency     1         1      1


## Normalized Term Frequency

The next functions attempt at converting the raw term frequency counts to some
sort of probability so that we can determine the probability a term appears in a
document.  Very basic normalization is done here, where the capitalization of
words is removed.

In [15]:
#Normalized Term Frequency
def termFrequency(term, document):
    normalizeDocument = document.lower().split()
    return normalizeDocument.count(term.lower()) / float(len(normalizeDocument))

def compute_normalizedtf(documents):
    tf_doc = []
    for txt in documents:
        sentence = txt.split()
        norm_tf= dict.fromkeys(set(sentence), 0)
        for word in sentence:
            norm_tf[word] = termFrequency(word, txt)
        tf_doc.append(norm_tf)
        df = pd.DataFrame([norm_tf])
        idx = 0
        new_col = ["Normalized TF"]    
        df.insert(loc=idx, column='Document', value=new_col)
        print(df)
    return tf_doc

tf_doc = compute_normalizedtf([doc1, doc2, doc3])

        Document  want  something  learning  life   to  charge   in  start  \
0  Normalized TF   0.1        0.1       0.1   0.1  0.2     0.1  0.1    0.1   

     I  
0  0.1  
        Document  about  something     no  reading   else   life    one  knows
0  Normalized TF  0.125      0.125  0.125    0.125  0.125  0.125  0.125  0.125
        Document      stop  learning     Never
0  Normalized TF  0.333333  0.333333  0.333333


# Inverse Document Frequency (IDF)

Here, we address the issue of large term counts having a disproportionate effect
in determining relevancy with respect to matching a particular query.  By
applying an inverse document frequency to the term frequency, we suppress the
weights of terms with high frequency counts.  Indeed, words that occur less
often are a better determinant of matching queries to a document.  Hence, we
apply the following formula to find the IDF:
$$ \operatorname{idf_t} = \log_{10}\left(\frac{N}{df_t}\right) $$
where $N$ is the total number of documents in the corpus and $df_t$ is the
number of documents in which the term $t$ appears.

In this case, it looks like the author increases the value for idf by $1$.  I am
not sure why the author does this since the formula doe not call for this
addition.  I have not removed the addition of $1$ because it does not change the
final results since this addition occurs in all cases.

In [44]:
def inverseDocumentFrequency(term, allDocuments):
    numDocumentsWithThisTerm = 0
    for doc in range (0, len(allDocuments)):
        if term.lower() in allDocuments[doc].lower().split():
            numDocumentsWithThisTerm = numDocumentsWithThisTerm + 1
 
    if numDocumentsWithThisTerm > 0:
        return 1.0 + math.log(float(len(allDocuments)) / numDocumentsWithThisTerm)
    else:
        return 1.0
    
def compute_idf(documents):
    idf_dict = {}
    for doc in documents:
        sentence = doc.split()
        for word in sentence:
            idf_dict[word] = inverseDocumentFrequency(word, documents)
    return idf_dict
    
idf_dict = compute_idf([doc1, doc2, doc3])

compute_idf([doc1, doc2, doc3])

{'I': 2.09861228866811,
 'want': 2.09861228866811,
 'to': 2.09861228866811,
 'start': 2.09861228866811,
 'learning': 1.4054651081081644,
 'charge': 2.09861228866811,
 'something': 1.4054651081081644,
 'in': 2.09861228866811,
 'life': 1.4054651081081644,
 'reading': 2.09861228866811,
 'about': 2.09861228866811,
 'no': 2.09861228866811,
 'one': 2.09861228866811,
 'else': 2.09861228866811,
 'knows': 2.09861228866811,
 'Never': 2.09861228866811,
 'stop': 2.09861228866811}

Using the IDF values above, the author then calculates the IDF score for each
word in the query.  The following function compares the query words to the IDF
dictionary and term frequency dictionary for each document to find the TF-IDF
score for each word in each document.

In [48]:
# tf-idf score across all docs for the query string("life learning")
def compute_tfidf_with_alldocs(documents , query):
    tf_idf = []
    index = 0
    query_tokens = query.split()
    df = pd.DataFrame(columns=['doc'] + query_tokens)
    for doc in documents:
        df['doc'] = np.arange(0 , len(documents))
        doc_num = tf_doc[index]
        sentence = doc.split()
        for word in sentence:
            for text in query_tokens:
                if(text == word):
                    idx = sentence.index(word)
                    tf_idf_score = doc_num[word] * idf_dict[word]
                    tf_idf.append(tf_idf_score)
                    df.iloc[index, df.columns.get_loc(word)] = tf_idf_score
        index += 1
    df.fillna(0 , axis=1, inplace=True)
    return tf_idf , df
            
documents = [doc1, doc2, doc3]
tf_idf , df = compute_tfidf_with_alldocs(documents , query)
print(df)

   doc      life  learning
0  0.0  0.140547  0.140547
1  1.0  0.175683  0.000000
2  2.0  0.000000  0.468488


# Cosine Similarity

The author takes an incremental approach here to calculate cosine similarities
between documents here.  They first calculate a normalized term frequency
dictionary, followed by calculating an IDF dictionary.  Once the term frequency
and IDF are calculated a final document weight vector can be calculated from the
results.

### Term Frequency Function

In [34]:
#Normalized TF for the query string("life learning")
def compute_query_tf(query):
    query_norm_tf = {}
    tokens = query.split()
    for word in tokens:
        query_norm_tf[word] = termFrequency(word , query)
    return query_norm_tf
query_norm_tf = compute_query_tf(query)
print(query_norm_tf)

{'life': 0.5, 'learning': 0.5}


### IDF Function

In [35]:
#idf score for the query string("life learning")
def compute_query_idf(query):
    idf_dict_qry = {}
    sentence = query.split()
    documents = [doc1, doc2, doc3]
    for word in sentence:
        idf_dict_qry[word] = inverseDocumentFrequency(word ,documents)
    return idf_dict_qry
idf_dict_qry = compute_query_idf(query)
print(idf_dict_qry)

{'life': 0.17609125905568124, 'learning': 0.17609125905568124}


### TF-IDF Function

In [36]:
#tf-idf score for the query string("life learning")
def compute_query_tfidf(query):
    tfidf_dict_qry = {}
    sentence = query.split()
    for word in sentence:
        tfidf_dict_qry[word] = query_norm_tf[word] * idf_dict_qry[word]
    return tfidf_dict_qry
tfidf_dict_qry = compute_query_tfidf(query)
print(tfidf_dict_qry)

{'life': 0.08804562952784062, 'learning': 0.08804562952784062}


## Cosine Similarity Function

Finally, all the above results can be combined to calculate cosine similarity
with the following formula:
$$ \cos(\vec{q},\vec{d}) = \frac{\vec{q} \cdot \vec{d}}{\lVert\vec{q}\rVert
\lVert\vec{d}\rVert} =  \frac{\displaystyle\sum_{i=1}^{\lvert V\rvert} q_i
d_i}{\sqrt{\displaystyle\sum_{i=1}^{\lvert V\rvert}
q_i^2}\sqrt{\displaystyle\sum_{i=1}^{\lvert V\rvert} di^2}}$$

The `cosine_similarity` function is pretty self-explanatory.  It simply adds up
the products of the query and document TF-IDF scores, then divides them by the
product of the norms of each.

I am not sure why the original author created a generator for the flatten
function since we don't really need to use lazy programming here: we are looking
to calculate all results.

In [37]:
#Cosine Similarity(Query,Document1) = Dot product(Query, Document1) / ||Query|| * ||Document1||

"""
Example : Dot roduct(Query, Document1) 

     life:
     = tfidf(life w.r.t query) * tfidf(life w.r.t Document1) +  / 
     sqrt(tfidf(life w.r.t query)) * 
     sqrt(tfidf(life w.r.t doc1))
     
     learning:
     =tfidf(learning w.r.t query) * tfidf(learning w.r.t Document1)/
     sqrt(tfidf(learning w.r.t query)) * 
     sqrt(tfidf(learning w.r.t doc1))

"""
def cosine_similarity(tfidf_dict_qry, df , query , doc_num):
    dot_product = 0
    qry_mod = 0
    doc_mod = 0
    tokens = query.split()
   
    for keyword in tokens:
        dot_product += tfidf_dict_qry[keyword] * df[keyword][df['doc'] == doc_num]
        #||Query||
        qry_mod += tfidf_dict_qry[keyword] * tfidf_dict_qry[keyword]
        #||Document||
        doc_mod += df[keyword][df['doc'] == doc_num] * df[keyword][df['doc'] == doc_num]
    qry_mod = np.sqrt(qry_mod)
    doc_mod = np.sqrt(doc_mod)
    #implement formula
    denominator = qry_mod * doc_mod
    cos_sim = dot_product/denominator
     
    return cos_sim

from collections.abc import Iterable
def flatten(lis):
     for item in lis:
        if isinstance(item, Iterable) and not isinstance(item, str):
             for x in flatten(item):
                yield x
        else:        
             yield item


In [50]:
def rank_similarity_docs(data):
    cos_sim =[]
    for doc_num in range(0 , len(data)):
        cos_sim.append(cosine_similarity(tfidf_dict_qry, df , query , doc_num).tolist())
    return cos_sim
similarity_docs = rank_similarity_docs(documents)
doc_names = ["Document1", "Document2", "Document3"]
print(doc_names)
print(list(flatten(similarity_docs)))

['Document1', 'Document2', 'Document3']
[1.0, 0.7071067811865476, 0.7071067811865476]


## Program_1

In [72]:
theonion = pd.read_csv("./file_archive/theonion.csv")
theonion = theonion.dropna()

In [73]:
onioncontentlist = theonion.loc[:, 'Content'].values.tolist()
onioncontentlist[:2]

['WASHINGTON, DC—According to an alarming new Department of Defense report combining civilian, military, and calendric evidence, Iran may be as few as nine years away from the year 2016. "Every day they get one day closer," Defense Secretary Robert Gates said during a White House press conference Tuesday. "At the rate they\'re going, they will reach 2016 at the same time as the United States—and given their geographic position relative to the international date line, possibly even sooner." The report recommended that the U.S. engage in bellicose international posturing, careless brinksmanship, and an eventual overwhelming series of nuclear strikes in order to prevent Iran from reaching this milestone.',
 'BOSTON—After living in residence halls during his first three semesters at the University of Massachusetts Dartmouth, sophomore student Dzhokar Tsarnaev was finally able to get a place of his own and move off campus this week, the 19-year-old told reporters. “Last semester I shared a 

In [74]:
tf_onion = compute_normalizedtf(onioncontentlist)
tf_onion[:2]

        Document     reach   nuclear        be        in  Secretary       one  \
0  Normalized TF  0.009259  0.009259  0.009259  0.018519   0.009259  0.009259   

     going,  closer,"    during  ...  relative  bellicose      they      will  \
0  0.009259  0.009259  0.009259  ...  0.009259   0.009259  0.018519  0.009259   

       time  Tuesday.     given  WASHINGTON,  careless      from  
0  0.009259  0.009259  0.009259     0.009259  0.009259  0.018519  

[1 rows x 89 columns]
        Document    shared        be       for     don’t        in     Hall,  \
0  Normalized TF  0.005319  0.005319  0.010638  0.005319  0.005319  0.005319   

   roommate,     which  reporters.  ...      Pine        is  required  \
0   0.005319  0.005319    0.005319  ...  0.005319  0.021277  0.005319   

      share    Dining   located      long       was       own      from  
0  0.005319  0.005319  0.005319  0.005319  0.005319  0.005319  0.005319  

[1 rows x 132 columns]
        Document   reports    snack. 

[{'reach': 0.009259259259259259,
  'nuclear': 0.009259259259259259,
  'be': 0.009259259259259259,
  'in': 0.018518518518518517,
  'Secretary': 0.009259259259259259,
  'one': 0.009259259259259259,
  'going,': 0.009259259259259259,
  'closer,"': 0.009259259259259259,
  'during': 0.009259259259259259,
  'and': 0.018518518518518517,
  'year': 0.009259259259259259,
  'military,': 0.009259259259259259,
  'new': 0.009259259259259259,
  'Defense': 0.018518518518518517,
  'nine': 0.009259259259259259,
  'brinksmanship,': 0.009259259259259259,
  'civilian,': 0.009259259259259259,
  '"Every': 0.009259259259259259,
  'years': 0.009259259259259259,
  'that': 0.009259259259259259,
  'as': 0.027777777777777776,
  'possibly': 0.009259259259259259,
  'strikes': 0.009259259259259259,
  'date': 0.009259259259259259,
  'at': 0.009259259259259259,
  'position': 0.009259259259259259,
  'even': 0.009259259259259259,
  'may': 0.009259259259259259,
  'an': 0.018518518518518517,
  'Iran': 0.018518518518518517,
